In [1]:
from dlisio import dlis
import pandas as pd
import numpy as np

def process_curve_info(channel_data):
    def df_column_uniquify(df):
        df_columns = df.columns
        new_columns = []
        print('AQUI')
        for item in df_columns:
            counter = 0
            newitem = item
            while newitem in new_columns:
                counter += 1
                newitem = "{}_{}".format(item, counter)
            new_columns.append(newitem)
        df.columns = new_columns
        return df

def dlis_access(filepath, depth_track = 0, logical_file = 0, null=-999.25):
    with dlis.load(filepath) as files:
        frame_all = {} # dicionario vazio (armazenar os logical files)
        ii = 0 # indexador (nome de acesso) dos logical files
        for f in files:
            for frame in f.frames:
                curves = frame.curves()

                channel_data = {
                        "curves_name": [],
                        "longs": [],
                        "unit": [],
                        "curves_L": [],
                        "las_units": [],
                        "las_longs": [],
                    }

                for channel in frame.channels:
                    channel_data["curves_name"].append(channel.name)
                    channel_data["longs"].append(channel.long_name)
                    channel_data["unit"].append(channel.units)
                    curves = channel.curves()
                    channel_data["curves_L"].append(curves)
                    
                print(channel_data["curves_name"])
                print(channel_data["curves_L"][depth_track])
                print()
                
                A = process_curve_info(channel_data)
                
                DATA = {}
                for i in range(len(A['curves_name'])):
                    DATA[A['curves_name'][i]] = {}
                    DATA[A['curves_name'][i]]['curve'] = A['curves_L'][i]
                    DATA[A['curves_name'][i]]['unit'] = A['unit'][i]

                frame_all[ii] = DATA
            ii += 1
            
    return frame_all[logical_file]

In [2]:
DATA = dlis_access(
    r'data\IODP_311-U1325A_lwd-images-proc.dlis',
    logical_file = 1, null=-888) # , null=-999.25

['TDEP', 'ROSI']
[  8.93064    8.961121   8.9916   ... 358.47516  358.5056   358.5361  ]



NameError: name 'process_curve_info' is not defined

In [44]:
#tosave = ['TDEP', 'BS', 'CS', 'MINM', 'TENS', 'DLIM', 'DPHD', 'DPHI', 'DPHS', 'DRHO', 'PE', 'RHOB', 'AHV', 'AHVT', 'BHV', 'BHVT', 'CALI', 'DCAL', 'GR', 'FDSN', 'NDSN', 'NLIM', 'NPHD', 'NPHI', 'NPHS', 'NRAT', 'TEM2', 'FBA', 'FDE', 'FHI', 'FLI', 'FLO', 'FPK', 'NBA', 'NDE', 'NHI', 'NLI', 'NLO', 'NPK', 'RT90', 'RT60', 'RT30', 'RT20', 'RT10', 'RT06', 'DT', 'ITT']
tosave = ['TDEP', 'BS', 'RHOB', 'GR','RT90','DT']


all_data = []
for m in tosave:  
    column = [m] + [DATA[m]['unit']] + list(DATA[m]['curve'])
    all_data.append(column)
    

DF = pd.DataFrame(all_data).T
new_header = DF.iloc[0]
DF = DF[1:]
DF.columns = new_header
DF.to_csv("C:\\Users\\isabela\\Desktop\\1-BRSA-1007-RJS\\Perfis_Digitais\\LAS\\1-brsa-1007-rjs_8_quad.csv", index=False)